# console capture and replay for notebooks

This notebook shows an example of an small utility that can help capture the output sent to stdout by a long-running process in a notebook.

In [1]:
from __future__ import print_function

from  dl_utils import console_capture as cc

A preliminary check: this cell should print

  `789...`

since the carriage return should make it overwrite all previous output. If not, this is possibly a [buggy notebook version](https://github.com/jupyter/notebook/issues/1970). Update notebook to at least 4.3.1


In [2]:
a = '123...\r456...\r789...'
print(a)

789...


Backspaces are not correctly processed by notebooks, though (they are ignored). So this one will not print what it should:

In [3]:
a = '123...\r456...\r789\b\b\babc...'
print(a)

789abc...


# High-level API

## Process example

To use the high-level API we need out processing task in the form of a [callable](https://docs.python.org/3/library/functions.html#callable)  (a function, method, or in general any object with a `__call__` [attribute](https://docs.python.org/3/reference/datamodel.html#object.__call__)), with arbitrary arguments

In [4]:
import time

def processing_call( num, msg, **kwargs ):
    '''
    A function that basically does nothing, but takes its time to do 
    that nothing and prints out progress reports while it is doing it
    '''
    tic = time.time()
    # Outer loop: 'num' times
    for j in range(num):
        # Inner loop: 10 times
        for i in range(10):
            # Print progress reports. Use control characters to overwrite the line
            print( "\b\b\b\b\b\b\b\b\b\b\b\r", '{:2} {:2} {:5.2f}'.format( j+1, i+1, time.time() - tic ), end=' ' )
            time.sleep(0.10)
        print('<iteration done>') # next line
        time.sleep(1)
    print( "Done! Msg={} Param={}".format(msg,kwargs) )

## Basic style
The basic procedure is creating a wrap object by passing it the callable with its required arguments ...

In [5]:
p = cc.ProcessWrap( processing_call, 4, "Hi", param="I'm a process" )

... and then calling the `process()` method on that object. It appears to be the same as executing the callable directly: the process is carried out, the cell appears as **`[*]`** (i.e. working) and the notebook as _busy_ and the progress results printed by the callable are written back to the notebook.

In [6]:
p.process()

Launching process ... 
  1 10  0.91 <iteration done>
  2 10  2.92 <iteration done>
  3 10  4.93 <iteration done>
  4 10  6.93 <iteration done>
Done! Msg=Hi Param={'param': "I'm a process"}


But behind the scenes, the printed data has been written to a temporal file, whose contents are automatically loaded upon termination as internal data object (and the file deleted), so they can be reprinted at will:

In [7]:
p.show()

----- DONE -----
  1 10  0.91 <iteration done>
  2 10  2.92 <iteration done>
  3 10  4.93 <iteration done>
  4 10  6.93 <iteration done>
Done! Msg=Hi Param={'param': "I'm a process"}


### accessing raw data

The same output is also available in raw form through the `data` attribute of the object:

In [8]:
p.data

'\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  1  0.00 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  2  0.10 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  3  0.20 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  4  0.30 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  5  0.40 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  6  0.50 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  7  0.61 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  8  0.71 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  9  0.81 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1 10  0.91 <iteration done>\n\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  2  1  2.01 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  2  2  2.11 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  2  3  2.21 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  2  4  2.31 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  2  5  2.41 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  2  6  2.51 \x08\x08\x08\x08\x08\x08\x08\x08\x08\

Obviously all those control characters make the data a bit difficult to read. We can use the provided `clean_string()` function, which applies backspaces (`\b`) and carriage returns (`\r`) as they would be processed were the data sent to standard output:

In [9]:
cc.clean_string( p.data )

'  1 10  0.91 <iteration done>\n  2 10  2.92 <iteration done>\n  3 10  4.93 <iteration done>\n  4 10  6.93 <iteration done>\nDone! Msg=Hi Param={\'param\': "I\'m a process"}\n'

### closing and reopening the notebook window

More importantly, the `process()` call, though it seems to be blocking it is doing so by actually looping over a check-sleep cycle. The actual process is being carried out in a **background thread**, which will continue saving its results to the temporal file even if the browser window is closed. 

Opening the notebook later on and executing the `show()` method will print out the results stored in the file. Moreover, it will reconnect the standard output of the process with the notebook cell, so that further output gets back to the notebook. We can test it with a longer running process:

In [10]:
p2 = cc.ProcessWrap( processing_call, 50, "Hi" )

In [ ]:
p2.process()  
# after launching execution of this cell, save the notebook & close the browser window 
# (confirm the warning about leaving the page)

Launching process ... 
  1 10  0.91 <iteration done>
  2 10  2.92 <iteration done>
  3 10  4.93 <iteration done>
  4  2  6.13 

In [12]:
# Then come back again here, INTERRUPT the kernel and execute this cell. 
# It should print all output received so far, and continue from there
p2.show() 

----- RUNNING -----
  1 10  0.91 <iteration done>
  2 10  2.92 <iteration done>
  3 10  4.93 <iteration done>
  4 10  6.94 <iteration done>
  5 10  8.95 <iteration done>
  6 10 10.96 <iteration done>
  7 10 12.97 <iteration done>
  8 10 14.97 <iteration done>
  9 10 16.98 <iteration done>
 10 10 18.99 <iteration done>
 11 10 21.00 <iteration done>
 12 10 23.01 <iteration done>
 13 10 25.02 <iteration done>
 14 10 27.03 <iteration done>
 15 10 29.04 <iteration done>
 16 10 31.05 <iteration done>
 17 10 33.05 <iteration done>
 18 10 35.06 <iteration done>
 19 10 37.07 <iteration done>
 20 10 39.08 

There is one caveat: as seen in the previous cell, the `process()` call was blocking the notebook (by doing the mentioned check-sleep cycle), so coming back to the notebook (opening its window again) will make it appear unresponsive. 

You will need to _interrupt_ the kernel to stop that `process()` cell so that it is possible to execute a `show()` cell. This is not problematic, since what it is being interrupted is the foreground process, not the background thread that it is doing the real processing. But nevertheless if it is known for sure that the window will be closed and reopened, it will be better to use the non-blocking call below.

## Non-blocking usage

This modality works the same way, but adding the `block=False` parameter to the `process()` method: 

In [13]:
pn = cc.ProcessWrap( processing_call, 50, "Look! NonBlocking!" )

In [14]:
pn.process( block=False )
# Now the notebook appears free to use (not busy), though the background thread 
# is still pushing text to the cell output
# We can save & close the notebook here. There may or may not be a "confirm" warning now.

Launching process ... 
  1 10  0.91 <iteration done>
  2 10  2.92 <iteration done>
  3 10  4.93 

In [15]:
# After coming back, in this modality it is no longer necessary to interrupt the kernel. 
# Just execute this cell to reconnect with the processing cell output
pn.show()

----- RUNNING -----
  1 10  0.91 <iteration done>
  2 10  2.92 <iteration done>
  3 10  4.93 <iteration done>
  4 10  6.94 <iteration done>
  5 10  8.94 <iteration done>
  6 10 10.95 <iteration done>
  7 10 12.97 <iteration done>
  8 10 14.98 <iteration done>
  9 10 16.99 <iteration done>
 10 10 19.00 <iteration done>
 11 10 21.00 <iteration done>
 12 10 23.01 <iteration done>
 13 10 25.02 <iteration done>
 14 10 27.03 <iteration done>
 15 10 29.05 <iteration done>
 16 10 31.06 <iteration done>
 17 10 33.07 <iteration done>
 18 10 35.08 <iteration done>
 19 10 37.09 <iteration done>
 20 10 39.10 <iteration done>
 21 10 41.11 

In [20]:
# We can still print the collected output, after it has finished
pn.show()

----- DONE -----
  1 10  0.93 <iteration done>
  2 10  2.96 <iteration done>
  3 10  4.99 <iteration done>
  4 10  7.02 <iteration done>
  5 10  9.05 <iteration done>
  6 10 11.07 <iteration done>
  7 10 13.10 <iteration done>
  8 10 15.13 <iteration done>
  9 10 17.19 <iteration done>
 10 10 19.20 <iteration done>
 11 10 21.23 <iteration done>
 12 10 23.26 <iteration done>
 13 10 25.29 <iteration done>
 14 10 27.31 <iteration done>
 15 10 29.34 <iteration done>
 16 10 31.37 <iteration done>
 17 10 33.39 <iteration done>
 18 10 35.42 <iteration done>
 19 10 37.45 <iteration done>
 20 10 39.48 <iteration done>
 21 10 41.50 <iteration done>
 22 10 43.52 <iteration done>
 23 10 45.55 <iteration done>
 24 10 47.58 <iteration done>
 25 10 49.61 <iteration done>
 26 10 51.64 <iteration done>
 27 10 53.67 <iteration done>
 28 10 55.70 <iteration done>
 29 10 57.73 <iteration done>
 30 10 59.75 <iteration done>
 31 10 61.77 <iteration done>
 32 10 63.80 <iteration done>
 33 10 65.83 <iteration

## Additional options

### verbose

Adding `verbose=False` to the `process()` method will suppress output (but still save to the temporal file)

In [16]:
pq = cc.ProcessWrap( processing_call, 3, "Now Quiet!" )
pq.process( verbose=False )
# This will take a few seconds, with no output

In [17]:
# We can still access the data
pq.show()

----- DONE -----
  1 10  0.90 <iteration done>
  2 10  2.91 <iteration done>
  3 10  4.91 <iteration done>
Done! Msg=Now Quiet! Param={}


### delete

The temporal file is automatically deleted upon task termination. This is usually not a problem because the data in the file is loaded onto the `ProcessWrap` object, and it can be printed via `show()` (or retrieved through the `data` attribute). But of course it will be lost if the notebook kernel is terminated. If the processing is very long or important, as a safety measure the `delete=False` parameter to `process()` will cancel file removal.

In [18]:
pk = cc.ProcessWrap( processing_call, 3, "Not deleted!" )
pk.process( delete=False )

Launching process ... 
  1 10  0.91 <iteration done>
  2 10  2.92 <iteration done>
  3 10  4.93 <iteration done>
Done! Msg=Not deleted! Param={}


In [19]:
# See the filename that was used
pk.logname

'/home/paulo/ML/IPNB/DL/notebook-Vgc932.log'

In [21]:
# The data is there for posterity
with open(pk.logname,"r") as f:
    print( f.read() )

  1 10  0.91 <iteration done>
  2 10  2.92 <iteration done>
  3 10  4.93 <iteration done>
Done! Msg=Not deleted! Param={}



In [22]:
# Or perhaps not
import os
os.unlink( pk.logname )

# Low level API
The `ProcessWrap` object functionality is implemented via lower level objects:

* `OutputDest` is a file-like object that substitutes _stdout_ & _stderr_ to perform the data capture
* `ConsoleCapture` provides an object that uses `OutputDest`, and `ConsoleCaptureCtx` a context manager for it
* `ProcessingThread` provides the processing thread that encapsulates its output via `ConsoleCaptureCtx`

These objects can be used directly if more specialised treatment is needed

### ConsoleCaptureCtx


In [4]:
import datetime

with cc.ConsoleCaptureCtx(verbose=False) as ll1:
    for i in range(20):
        print( '{:3}  {}'.format(i+1, datetime.datetime.now()) )

In [5]:
ll1.reprint()

  1  2016-12-23 20:44:56.716750
  2  2016-12-23 20:44:56.716888
  3  2016-12-23 20:44:56.716934
  4  2016-12-23 20:44:56.716964
  5  2016-12-23 20:44:56.716990
  6  2016-12-23 20:44:56.717015
  7  2016-12-23 20:44:56.717041
  8  2016-12-23 20:44:56.717066
  9  2016-12-23 20:44:56.717091
 10  2016-12-23 20:44:56.717116
 11  2016-12-23 20:44:56.717216
 12  2016-12-23 20:44:56.717250
 13  2016-12-23 20:44:56.717276
 14  2016-12-23 20:44:56.717301
 15  2016-12-23 20:44:56.717326
 16  2016-12-23 20:44:56.717351
 17  2016-12-23 20:44:56.717376
 18  2016-12-23 20:44:56.717401
 19  2016-12-23 20:44:56.717427
 20  2016-12-23 20:44:56.717452


In [6]:
ll1.data

'  1  2016-12-23 20:44:56.716750\n  2  2016-12-23 20:44:56.716888\n  3  2016-12-23 20:44:56.716934\n  4  2016-12-23 20:44:56.716964\n  5  2016-12-23 20:44:56.716990\n  6  2016-12-23 20:44:56.717015\n  7  2016-12-23 20:44:56.717041\n  8  2016-12-23 20:44:56.717066\n  9  2016-12-23 20:44:56.717091\n 10  2016-12-23 20:44:56.717116\n 11  2016-12-23 20:44:56.717216\n 12  2016-12-23 20:44:56.717250\n 13  2016-12-23 20:44:56.717276\n 14  2016-12-23 20:44:56.717301\n 15  2016-12-23 20:44:56.717326\n 16  2016-12-23 20:44:56.717351\n 17  2016-12-23 20:44:56.717376\n 18  2016-12-23 20:44:56.717401\n 19  2016-12-23 20:44:56.717427\n 20  2016-12-23 20:44:56.717452\n'

### ConsoleCapture

In [15]:
ll2 = cc.ConsoleCapture()

In [16]:
print( "A")
print( "B" )

A
B


In [17]:
ll2.start()

print( "CDE...\r", end='')
print( "...FGH")
print( "IJK")

ll2.stop();

...FGH
IJK


In [18]:
ll2.reprint().remove();

...FGH
IJK
